In [3]:
from ultralytics import YOLO
import cv2
import os

# Load fine-tuned weights for detection and classification
detector = YOLO('runs/detect/train35/weights/best.pt')    # Detection model
classifier = YOLO('runs/classify/train20/weights/best.pt') # Classification model

# Thresholds
CONF_THRESHOLD = 0.5  # min confidence to keep a box


def process_image(image_path, output_dir=None):
    print(f"[INFO] Processing image: {image_path}")

    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"[ERROR] Failed to load image at path: {image_path}")
        return

    # Detection
    det_results = detector.predict(
        source=image_path,
        augment=True,
        conf=CONF_THRESHOLD,   # only get boxes above threshold
        iou=0.45,
        verbose=False
    )
    if not det_results:
        print(f"[WARNING] No detections above threshold for {image_path}")
        return

    boxes = det_results[0].boxes
    print(f"[INFO] Found {len(boxes)} detections >= {CONF_THRESHOLD:.2f} in {os.path.basename(image_path)}")

    # Prepare canvases
    det_vis = image.copy()
    cls_vis = image.copy()
    hybrid_vis = image.copy()
    high_conf_image = False

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        det_conf = float(box.conf[0])
        det_label = detector.names[int(box.cls[0])] if hasattr(box, 'cls') and box.cls is not None else 'obj'

        # ROI for classification
        roi = image[y1:y2, x1:x2]
        cls_conf = 0.0
        cls_label = ''
        if roi.size > 0:
            try:
                cls_res = classifier.predict(source=roi, imgsz=224, verbose=False)
                if cls_res and hasattr(cls_res[0], 'probs'):
                    cls_conf = float(cls_res[0].probs.top1conf)
                    cls_label = classifier.names[int(cls_res[0].probs.top1)]
            except Exception as e:
                print(f"[ERROR] Classification failed for ROI {i}: {e}")

        # Skip boxes if neither detection nor classification meets threshold
        if max(det_conf, cls_conf) < CONF_THRESHOLD:
            continue

        # Draw detection-only
        cv2.rectangle(det_vis, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(det_vis, f"{det_label}({det_conf:.2f})", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        # Draw classification-only if available
        if cls_conf >= CONF_THRESHOLD:
            cv2.rectangle(cls_vis, (x1, y1), (x2, y2), (255,255,0), 2)
            cv2.putText(cls_vis, f"{cls_label}({cls_conf:.2f})", (x1, y2+20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2)

        # Choose higher confidence for hybrid
        if cls_conf > det_conf:
            chosen_label = f"{cls_label}({cls_conf:.2f})"
            box_color = (255,0,0)
            chosen_conf = cls_conf
        else:
            chosen_label = f"{det_label}({det_conf:.2f})"
            box_color = (0,255,0)
            chosen_conf = det_conf

        if chosen_conf >= CONF_THRESHOLD:
            high_conf_image = True

        # Draw hybrid
        cv2.rectangle(hybrid_vis, (x1, y1), (x2, y2), box_color, 2)
        cv2.putText(hybrid_vis, chosen_label, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)

    # Display views
    cv2.imshow("Detections >= threshold", det_vis)
    cv2.imshow("Classification >= threshold", cls_vis)
    cv2.imshow("Hybrid Selection", hybrid_vis)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save if requested
    if output_dir:
        high_dir = os.path.join(output_dir, 'high_conf')
        low_dir = os.path.join(output_dir, 'low_conf')
        os.makedirs(high_dir if high_conf_image else low_dir, exist_ok=True)
        base = os.path.splitext(os.path.basename(image_path))[0]
        dest = high_dir if high_conf_image else low_dir
        cv2.imwrite(os.path.join(dest, f"{base}_det.png"), det_vis)
        cv2.imwrite(os.path.join(dest, f"{base}_cls.png"), cls_vis)
        cv2.imwrite(os.path.join(dest, f"{base}_hyb.png"), hybrid_vis)
        print(f"[INFO] Saved to {'high_conf' if high_conf_image else 'low_conf'} folder.")

if __name__ == '__main__':
    root = 'Road_Damage-OBJD/val'
    images_dir = os.path.join(root, 'images')
    output_dir = os.path.join(root, 'outputs')
    for fname in os.listdir(images_dir):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            process_image(os.path.join(images_dir, fname), output_dir)
    print("[INFO] Batch processing complete.")


[INFO] Processing image: Road_Damage-OBJD/val\images\01 - Chuck HolePunch Out (6).jpg
[INFO] Found 13 raw detections in 01 - Chuck HolePunch Out (6).jpg
[INFO] Saved to high_conf folder.
[INFO] Processing image: Road_Damage-OBJD/val\images\01 - Chuck-holePunch-out (RL).jpg
[INFO] Found 7 raw detections in 01 - Chuck-holePunch-out (RL).jpg
[INFO] Saved to high_conf folder.
[INFO] Processing image: Road_Damage-OBJD/val\images\01 - Patholes.jpg
[INFO] Found 2 raw detections in 01 - Patholes.jpg
[INFO] Saved to low_conf folder.
[INFO] Processing image: Road_Damage-OBJD/val\images\01 - pothole (4).jpg
[INFO] Found 9 raw detections in 01 - pothole (4).jpg
[INFO] Saved to high_conf folder.
[INFO] Processing image: Road_Damage-OBJD/val\images\01 - Pothole Bowl-shaped (LL).jpg
[INFO] Found 5 raw detections in 01 - Pothole Bowl-shaped (LL).jpg
[INFO] Saved to high_conf folder.
[INFO] Processing image: Road_Damage-OBJD/val\images\01 - POTHOLES (1).jpg
[INFO] Found 10 raw detections in 01 - POTHOL

In [26]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt

# Load fine-tuned weights for classification
classifier = YOLO('fold_1/results/weights/best.pt')  # Classification model

# Thresholds
CONF_THRESHOLD = 0.5  # min confidence to keep a box

def process_image_for_classification(image_path, output_dir=None, show_inline=True):
    print(f"[INFO] Processing image: {image_path}")

    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"[ERROR] Failed to load image at path: {image_path}")
        return

    # Prepare canvas
    cls_vis = image.copy()
    high_conf_image = False

    # Process image for classification
    try:
        cls_res = classifier.predict(source=image, imgsz=224, verbose=False)
        if cls_res and hasattr(cls_res[0], 'probs'):
            cls_conf = float(cls_res[0].probs.top1conf)
            cls_label = classifier.names[int(cls_res[0].probs.top1)]
        else:
            cls_conf = 0.0
            cls_label = 'Unknown'

        # Only process if confidence is above threshold
        if cls_conf >= CONF_THRESHOLD:
            high_conf_image = True
            cv2.putText(
                cls_vis,
                f"{cls_label} ({cls_conf:.2f})",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2
            )
    except Exception as e:
        print(f"[ERROR] Classification failed for image {image_path}: {e}")

    # Display inline in Jupyter
    if show_inline:
        # Convert BGR to RGB
        cls_vis_rgb = cv2.cvtColor(cls_vis, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(6, 6))
        plt.imshow(cls_vis_rgb)
        plt.title(f"{cls_label} ({cls_conf:.2f})")
        plt.axis('off')
        plt.show()

    # Save if requested
    if output_dir:
        suffix = 'high_conf' if high_conf_image else 'low_conf'
        output_folder = os.path.join(output_dir, suffix)
        os.makedirs(output_folder, exist_ok=True)
        base = os.path.splitext(os.path.basename(image_path))[0]
        save_path = os.path.join(output_folder, f"{base}_cls.png")
        cv2.imwrite(save_path, cls_vis)
        print(f"[INFO] Saved to {save_path}")

if __name__ == '__main__':
    root = 'Road_Damage/test'
    output_dir = os.path.join(root, 'outputs')

    # Loop through the subfolders for each class
    for class_folder in os.listdir(root):
        class_folder_path = os.path.join(root, class_folder)
        
        # Ensure it's a directory
        if os.path.isdir(class_folder_path):
            images_dir = os.path.join(class_folder_path, 'images')
            
            # Loop through the images in each class folder
            for fname in os.listdir(images_dir):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(images_dir, fname)
                    process_image_for_classification(
                        image_path,
                        output_dir=output_dir,
                        show_inline=True  # set False if you don't want to display
                    )
    print("[INFO] Batch processing complete.")



image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\test image\crack.png: 448x640 1 crack_issues, 73.9ms
Speed: 2.2ms preprocess, 73.9ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
